# 1 Frames

In [294]:
import robotic as ry
import numpy as np
import time

pi = np.pi
# some functions I wrote to convert rotation in an axis to quaternion
 
# General form -> quaternion=[cos(θ/2), xsin(θ/2), ysin(θ/2), zsin(θ/2)]
  
def quat_rotate_x(theta):
  # Creates a quaternion for rotation by angle theta around the X-axis: scalar= θ/2 and xyz = [1, 0, 0]
  return [np.cos(theta / 2), np.sin(theta / 2), 0, 0]

def quat_rotate_y(theta):
  # Creates a quaternion for rotation by angle theta around the X-axis: scalar= θ/2 and xyz = [1, 0, 0]
  return [np.cos(theta / 2), 0, np.sin(theta / 2), 0]

def quat_rotate_z(theta):
  # Creates a quaternion for rotation by angle theta around the Z-axis: scalar= θ/2 and xyz = [0, 0, 1]
  return [np.cos(theta / 2), 0, 0, np.sin(theta / 2)]

def quaternion_rotation_axis(theta, axis):
  # Creates a quaternion for rotation by angle theta around an arbitrary axis [x, y, z].
  
  # axis is given as unit vectors in xyz. e.g. rotation around x-axis will have axis [1, 0, 0]
  axis_length = np.sqrt(np.sum(np.square(axis)))
  unit_axis = [component / axis_length for component in axis]
  
  return [np.cos(theta / 2)] + [np.sin(theta / 2) * component for component in unit_axis]

quat_rotate_z(np.pi/4)


[0.9238795325112867, 0, 0, 0.3826834323650898]

### Part A

In [295]:
C = ry.Config()
C.addFile(ry.raiPath("../rai-robotModels/objects/kitchen.g"))
C.view()

# Create hammer. Child of sink1
hammer = C.addFrame(name="hammer", parent="sink1")

# Hammer obj1 - handle. Child of hammer
handle = C.addFrame(name="handle", parent="hammer")

# Hammer obj2 - head. Child of handle
head = C.addFrame(name="head", parent="handle")

In [296]:
# set props
# position the handle [0.1, 0.1, 0.415] according to ’sink1’.
handle.setShape(ry.ST.ssBox, size=[ 0.3, 0.05, 0.03, 0.005 ]) \
  .setColor( [0.65, 0.5, 0.39] ) \
  .setRelativePosition( [0.1, 0.1, 0.415] )
  
head.setShape(ry.ST.ssBox, size=[ 0.1, 0.05, 0.03, 0.005 ]) \
  .setColor( [0.55, 0.57, 0.58] ) \
  .setRelativePosition( [0.125, 0.0 , 0.0] ) \
  .setQuaternion( [0, 1, 1, 0] )
    
    
# head.setQuaternion(quat_rotate_z(pi / 2))
C.view()

0

In [297]:
# testing, ignore
print("Handle position: \t", handle.getPosition())
print("Handle rel position: \t", handle.getRelativePosition())
print("Handle quaternion: \t", handle.getQuaternion())
print()
print("sink1 position: \t", C.getFrame("sink1").getPosition())
print("sink1 quaternion: \t", C.getFrame("sink1").getQuaternion())
print()
print("Hammer position: \t", hammer.getPosition())
print("Hammer rel position: \t", hammer.getRelativePosition())

Handle position: 	 [-0.4    2.1    0.815]
Handle rel position: 	 [0.1   0.1   0.415]
Handle quaternion: 	 [1. 0. 0. 0.]

sink1 position: 	 [-0.5  2.   0.4]
sink1 quaternion: 	 [1. 0. 0. 0.]

Hammer position: 	 [-0.5  2.   0.4]
Hammer rel position: 	 [0. 0. 0.]


### Part B

In [298]:
# add mug obj, 
# size: [0.05, 0.05, 0.2, .01]
# color: [ 0.2, 0.2 ,0.2 ]
# relPos: [-0.1,-0.1, 0.5]
mug = C.addFrame(name="mug", parent="sink1")
mug.setShape(ry.ST.ssBox, size=[ 0.05, 0.05, 0.2, 0.01 ]) \
  .setColor([ 0.2, 0.2 ,0.2 ]) \
  .setRelativePosition( [ -0.1, -0.1, 0.5 ] )

# add tray obj, 
# size: [0.2, 0.2, 0.05, 0.02]
# color: [ 0, 1, 0 ]
# relPos: [0, 0, 0.42]
# quat: [0.9238795325112867, 0, 0, 0.3826834323650898]

tray = C.addFrame(name="tray", parent="stove1")
tray.setShape(type=ry.ST.ssBox, size=[0.2, 0.2, 0.05, 0.02]) \
  .setColor([ 0, 1, 0 ]) \
  .setRelativePosition( [ 0, 0, 0.42 ] ) \
  .setQuaternion( quat_rotate_z( pi/4 ) )   # rotated in z-axis by pi/4 OR 45 degrees
  
C.view()
print("Quaternion for the rotation in z axis: ", quat_rotate_z( pi/4 ))

Quaternion for the rotation in z axis:  [0.9238795325112867, 0, 0, 0.3826834323650898]


### Part C

In [299]:
# make the tray bigger to carry both of the items
trayCurSize = tray.getSize()
print(trayCurSize)

[0.2  0.2  0.05 0.02]


In [300]:
tray.setShape(type=ry.ST.ssBox, size=( [ 0.4, 0.4, 0.05, 0.02 ] ))

C.view()

0

In [301]:
# first remove hammer's old parent, i.e. sink1, and mug's ld parent, i.e. stove1
hammer.unLink()
mug.unLink()

# reset positions (or rather pose, but orientation remained unchanged so no need to change it here)
hammer.setPosition([0, 0, 0])
handle.setPosition([0, 0, 0])
mug.setPosition([0, 0, 0])

In [302]:
# Now set tray as new parent of hammer and mug
hammer.setParent(tray, False, False)
mug.setParent(tray, False, False)

C.view()

0

In [303]:
# now re-adjust positions of mug and hammer as before
# Also, raise their z axis so that they're just above the tray surface
mug_midpoint_z = 0.2 / 2
hammer_midpoint_z = 0.03 / 2
tray_midpoint_z = 0.05 / 2
stove1_midpoint_z = 0.4 / 2
mug.setRelativePosition( [-0.1, -0.1, mug_midpoint_z + tray_midpoint_z ] )
hammer.setRelativePosition( [ (0.1/0.6) * 0.4 , (0.1/0.6) * 0.4 , hammer_midpoint_z + tray_midpoint_z ] )

# State1
C.view()

0

In [304]:
# now place tray above table1
tray.unLink()

# get table1 frame
table1_f = C.getFrame("table1")
tray.setParent(table1_f, False, False) \
  .setRelativePosition( [0, 0, 0.42] ) \
  .setQuaternion( quat_rotate_z( 3 * np.pi/4 ) )


# tray.setShape(type=ry.ST.ssBox, size=(tray.getSize() * 2))

C.view()

0

In [307]:
print("Hammer position rel to tray: ", hammer.getRelativePosition())
print("Hammer position: ", hammer.getPosition())

print("Mug position rel to tray: ", mug.getRelativePosition())
print("Mug position: ", mug.getPosition())


Hammer position rel to tray:  [0.06666667 0.06666667 0.04      ]
Hammer position:  [1.90571910e+00 1.38777878e-17 8.45000000e-01]
Mug position rel to tray:  [-0.1   -0.1    0.125]
Mug position:  [ 2.14142136e+00 -1.38777878e-17  9.30000000e-01]


### Part D

In [218]:
origsize = mug.getSize()
curMugPos = mug.getPosition()
print(curMugPos)
print("Tray relPos to table1: ", tray.getRelativePosition())
print("Tray quat to table1: ", tray.getRelativeQuaternion())
print()

print("Mug relPos to tray: ", mug.getRelativePosition())
print("Mug quat to tray: ", mug.getRelativeQuaternion())
print()


[ 2.14 -0.02  0.94]
Tray relPos to table1:  [0.   0.   0.43]
Tray quat to table1:  [-0.9486833   0.          0.         -0.31622777]

Mug relPos to tray:  [-0.1   -0.1    0.125]
Mug quat to tray:  [1. 0. 0. 0.]



In [219]:
sink1 = C.getFrame("sink1")
sink1.getPosition()


mug.unLink()
mug.setParent(sink1, False, False)
mug.setRelativePosition([ 0, 0, 0.5 ])
# mug.setShape(ry.ST.ssBox, size=(origsize * 1))
C.view()

0

# 2 Two Link Manipulator

In [16]:
import robotic as ry
import numpy as np
import time

pi = np.pi
# some functions I wrote to convert rotation in an axis to quaternion
 
# General form -> quaternion=[cos(θ/2), xsin(θ/2), ysin(θ/2), zsin(θ/2)]
  
def quat_rotate_x(theta):
  # Creates a quaternion for rotation by angle theta around the X-axis: scalar= θ/2 and xyz = [1, 0, 0]
  return [np.cos(theta / 2), np.sin(theta / 2), 0, 0]

def quat_rotate_y(theta):
  # Creates a quaternion for rotation by angle theta around the X-axis: scalar= θ/2 and xyz = [1, 0, 0]
  return [np.cos(theta / 2), 0, np.sin(theta / 2), 0]

def quat_rotate_z(theta):
  # Creates a quaternion for rotation by angle theta around the Z-axis: scalar= θ/2 and xyz = [0, 0, 1]
  return [np.cos(theta / 2), 0, 0, np.sin(theta / 2)]

def quaternion_rotation_axis(theta, axis):
  # Creates a quaternion for rotation by angle theta around an arbitrary axis [x, y, z].
  
  # axis is given as unit vectors in xyz. e.g. rotation around x-axis will have axis [1, 0, 0]
  axis_length = np.sqrt(np.sum(np.square(axis)))
  unit_axis = [component / axis_length for component in axis]
  
  return [np.cos(theta / 2)] + [np.sin(theta / 2) * component for component in unit_axis]

quat_rotate_z(np.pi/2)


[0.7071067811865476, 0, 0, 0.7071067811865475]

### Part A
Modified .g file is hw1_two_link_manipulator.g

In [17]:
C = ry.Config()
C.addFile("hw1_two_link_manipulator.g")
C.view()

0

### Part B

In [33]:
target = C.getFrame("target")
# initialise joint angles to random
joint_angles = 2 * np.pi * np.random.rand(3)
def forward_kinematics(q):
  # get the random angles
  q0 = q[0]
  q1 = q[1]
  
  # Method 1: 
  y_P_blue_red = np.sin(q0)
  z_P_blue_red = np.cos(q0)
  
  y_P_green_blue = np.sin(q0) * np.cos(q1) + np.cos(q0) * np.sin(q1)
  z_P_green_blue = np.cos(q0) * np.cos(q1) - np.sin(q0) * np.sin(q1)
  
  # now to get pos of red rel to green, we add pos of red rel to blue with pos of blue to green
  y_P_green_red = y_P_blue_red + y_P_green_blue
  z_P_green_red = z_P_blue_red + z_P_green_blue
  # return np.array( [0, y_P_green_red, z_P_green_red] )
  
  # Method 2
  y = np.sin(q0) + np.sin(q0+q1)
  z = np.cos(q0) + np.cos(q0+q1)
  return ([ 0, y, z ])


# this is the target that my FK function has output for the current iteration
pos_target = forward_kinematics(joint_angles)
# ok so over here we place the target at the position produced by my FK function
target.setPosition( pos_target )
# set joint_angles
C.setJointState( joint_angles )
C.view()
print("Pos_target: ", pos_target)
print("Joint angles: ", joint_angles)

Pos_target:  [0, 0.015916560997032314, -0.041471782558546166]
Joint angles:  [4.32371971 3.18601754 3.04193417]


In [15]:
# Restart joint congif gor next run
q = np.zeros(C.getJointDimension())
C.setJointState(q)
target.setPosition( [0,0,0] )

### Part C

In [53]:
def Jacobian(q):
    q0 = q[0]
    q1 = q[1]

    # y = sin(q0) + sin(q0+q1) = sin(q0) + sin(q0)*cos(q1) + sin(q1)*cos(q0)
    # z = cos(q0) + cos(q0+q1) = cos(q0) + cos(q0)*cos(q1) - sin(q0)*sin(q1)

    j = np.zeros((2, 2))

    j[0][0] = np.cos(q0) + np.cos(q0)*np.cos(q1) - np.sin(q0)*np.sin(q1)
    j[0][1] = -np.sin(q0)*np.sin(q1) + np.sin(q1)*np.cos(q0)
    j[1][0] = -np.sin(q0) - np.sin(q0)*np.cos(q1) - np.cos(q0)*np.sin(q1)
    j[1][1] = -np.sin(q1)*np.cos(q0) - np.cos(q1)*np.sin(q0)

    return j

In [54]:
print(Jacobian([1.95205226, 3.15753276, 0]))

[[ 0.01474768  0.0207258 ]
 [-0.00604877  0.92214929]]


### Part D

(Kinematic) Singularities at tan(q0 + q1) = tan(q0), i.e. <br>
1. arm is fully extended (q1 = 0) OR 
2. arm is folded at jointB (q1 = pi)

# 3 Spatial Velocity for Moving Frame